In [1]:
import torch
from transformers import BertModel, BertTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader

# GPUが利用可能であれば使用
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# BERTモデルとトークナイザーのロード
model_name = "bert-large-uncased"
model = BertModel.from_pretrained(model_name).to(device)
tokenizer = BertTokenizer.from_pretrained(model_name)

# WikiTextデータセットのロード
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# バッチ処理のためにデータローダーを設定
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

def embed_batch(batch):
    texts = batch["text"]
    # トークナイズし、最大シーケンス長にパディング
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # BERTモデルで文の埋め込みを取得
    with torch.no_grad():
        outputs = model(**inputs)
    
    # プールされた出力 (通常は[CLS]トークンの出力) を取得
    sentence_embeddings = outputs.pooler_output
    return sentence_embeddings.cpu()

# バッチごとに文埋め込みを計算し、保存
all_embeddings = []
for batch in dataloader:
    embeddings = embed_batch(batch)
    all_embeddings.append(embeddings)

# リストをTensorに変換
all_embeddings = torch.cat(all_embeddings)

# 埋め込みを保存する (例: torch.save で保存)
torch.save(all_embeddings, "wikitext_bert_embeddings.pt")

print(f"Total embeddings: {all_embeddings.size(0)}, Embedding dimension: {all_embeddings.size(1)}")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/home/shinzato/git/text_power_raw/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Total embeddings: 36718, Embedding dimension: 1024
